# AI tool to add comments to the provided Java code

Here we build a Gradio App that uses the frontier models to add comments to a java code. For testing purposes I have used the *cheaper* versions of the models, not the ones the leaderboards indicate as the best ones.

In [0]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
import anthropic
import gradio as gr

In [0]:
# environment

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [0]:
openai = OpenAI()
claude = anthropic.Anthropic()
genai.configure()

In [0]:
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"
GEMINI_MODEL = 'gemini-2.0-flash-lite'

In [0]:
system_message = "You are an assistant that adds comments to java code. "
system_message += "Do not make any changes to the code itself."
system_message += "Use comments sparingly. Only add them in places where they help to undestand how the code works. Do not comment every single line of the code."

In [0]:
def user_prompt_for(code):
    user_prompt = "Add helpful comments to this java code. "
    user_prompt += "Do not change the code itself.\n\n"
    user_prompt += code
    return user_prompt

In [0]:
test_code = """
package com.hma.kafkaproducertest.producer;

import com.hma.kafkaproducertest.model.TestDTO;
import org.springframework.cloud.stream.function.StreamBridge;
import org.springframework.messaging.Message;
import org.springframework.messaging.support.MessageBuilder;
import org.springframework.stereotype.Component;

import java.util.Arrays;
import java.util.Comparator;
import java.util.StringJoiner;
import java.util.stream.Collectors;
import java.util.stream.IntStream;

@Component
public class TestProducer {

    public static final String EVENT_TYPE_HEADER = "event-type";
    private static final String BINDING_NAME = "testProducer-out-0";

    private final StreamBridge streamBridge;

    public TestProducer(StreamBridge streamBridge) {
        this.streamBridge = streamBridge;
    }

    public void sendMessage(TestDTO payload, String eventType){
        Message<TestDTO> message = MessageBuilder
                .withPayload(payload)
                .setHeader(EVENT_TYPE_HEADER, eventType)
                .build();

        streamBridge.send(BINDING_NAME, message);
    }

    public void test(String t1, String t2) {
        var s = t1.length() > t2.length() ? t2 : t1;
        var l = t1.length() > t2.length() ? t1 : t2;
        var res = true;
        for (int i = 0; i < s.length(); i++) {
            if (s.charAt(i) == l.charAt(i)) {
                res = false;
                break;
            }
        }
        System.out.println(res);
    }
}
"""

In [0]:
def stream_gpt(code):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(code)}
      ]
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [0]:
def stream_claude(code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[
            {"role": "user", "content": user_prompt_for(code)},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [0]:
def stream_gemini(code):
    gemini = genai.GenerativeModel(
        model_name=GEMINI_MODEL,
        system_instruction=system_message
    )
    stream = gemini.generate_content(user_prompt_for(code), stream=True)
    result = ""
    for chunk in stream:
        result += chunk.text or ""
        yield result

In [0]:
def comment_code(code, model):
    if model=="GPT":
        result = stream_gpt(code)
    elif model=="Claude":
        result = stream_claude(code)
    elif model=="Gemini":
        result = stream_gemini(code)
    else:
        raise ValueError("Unknown model")
    yield from result

In [0]:
with gr.Blocks() as ui:
    with gr.Row():
        original_code = gr.Textbox(label="Java code:", lines=10, value=test_code)
        commented_code = gr.Markdown(label="Commented code:")
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")
        comment = gr.Button("Comment code")

    comment.click(comment_code, inputs=[original_code, model], outputs=[commented_code])

ui.launch(inbrowser=True)

In [0]:
ui.close()

## Conclusion

In my personal opinion, at least when using these *cheaper* versions of the models, the result provided by Claude is the best. ChatGPT adds way too many comments even if the system message discourages that. Gemini provides a good result also, but maybe adds a tad too few comments -- although that certainly depends on your personal preferences.